# Test comparison across models

## Import Libreries and Data

You can run the notebook in two ways:

1. **Google Colab**: place the project folder `heat-forecast` in **MyDrive**. The setup cell below will mount Drive and automatically add `MyDrive/heat-forecast/src` to `sys.path` so `import heat_forecast` works out of the box.

2. **Local machine**:

   * **Installing our package:** from the project root, run `pip install -e .` once (editable install). Then you can open the notebook anywhere and import the package normally.
   * **Alternative:** if you’re running the notebook from `.../heat-forecast/notebooks/` without installing the package, the setup cell will detect `../src` and automatically add it to `sys.path`.

In [ ]:
# === Do not edit below ===
# --- Detect if running on Google Colab & Set base dir ---
# %cd /home/giovanni.lombardi/heat-forecast/notebooks
import subprocess
from pathlib import Path
import sys

def in_colab() -> bool:
    try:
        import google.colab  # type: ignore
        return True
    except Exception:
        return False

# Install required packages only if not already installed
def pip_install(pkg: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# Set base directory and handle environment
if in_colab():
    # Make sure IPython is modern (avoids the old %autoreload/imp issue if you ever use it)
    pip_install("ipython>=8.25")
    pip_install("ipykernel>=6.29")
    
    def install(package):
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    for pkg in ["statsmodels", "statsforecast", "mlforecast"]:
        pip_install(pkg)

    # Mount Google Drive
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')

    # Set base directory to your Drive project folder
    BASE_DIR = Path('/content/drive/MyDrive/heat-forecast')

    # Add `src/` to sys.path for custom package imports
    SRC_PATH = BASE_DIR / 'src'
    if str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))

    # Sanity checks (helpful error messages if path is wrong)
    assert SRC_PATH.exists(), f"Expected '{SRC_PATH}' to exist. Fix BASE_DIR."
    pkg_dir = SRC_PATH / "heat_forecast"
    assert pkg_dir.exists(), f"Expected '{pkg_dir}' package directory."
    init_file = pkg_dir / "__init__.py"
    assert init_file.exists(), f"Missing '{init_file}'. Add it so Python treats this as a package."

else:
    # Local: either rely on editable install (pip install -e .) or add src/ when running from repo
    # Assume notebook lives in PROJECT_ROOT/notebooks/
    BASE_DIR = Path.cwd().resolve().parent
    SRC_PATH = BASE_DIR / "src"

    added_src = False
    if (SRC_PATH / "heat_forecast").exists() and str(SRC_PATH) not in sys.path:
        sys.path.append(str(SRC_PATH))
        added_src = True

# --- Logging setup ---
import logging
from zoneinfo import ZoneInfo
from datetime import datetime

LOG_DIR  = (BASE_DIR / "logs")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / "run.log"
PREV_LOG = LOG_DIR / "run.prev.log"

# If there's a previous run.log with content, archive it to run.prev.log
if LOG_FILE.exists() and LOG_FILE.stat().st_size > 0:
    try:
        # Replace old run.prev.log if present
        if PREV_LOG.exists():
            PREV_LOG.unlink()
        LOG_FILE.rename(PREV_LOG)
    except Exception as e:
        # Fall back to truncating if rename fails (e.g., file locked)
        print(f"[warn] Could not archive previous log: {e}. Truncating current run.log.")
        LOG_FILE.write_text("")

# Configure logging: fresh file for this run + echo to notebook/stdout
file_handler   = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
stream_handler = logging.StreamHandler(sys.stdout)

fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(message)s",
                        datefmt="%m-%d %H:%M:%S")
file_handler.setFormatter(fmt)
stream_handler.setFormatter(fmt)

root = logging.getLogger()
root.handlers[:] = [file_handler, stream_handler]  # replace handlers (important in notebooks)
root.setLevel(logging.INFO)

# Use Rome time
logging.Formatter.converter = lambda *args: datetime.now(ZoneInfo("Europe/Rome")).timetuple()

logging.captureWarnings(True)
logging.info("=== Logging started (fresh current run) ===")
logging.info("Previous run (if any): %s", PREV_LOG if PREV_LOG.exists() else "none")

if added_src:
    logging.info("heat_forecast not installed; added src/ to sys.path")
else:
    logging.info("heat_forecast imported without modifying sys.path (likely installed)")

OPTUNA_DIR = BASE_DIR / "results" / "finetuning" / "lstm"
OPTUNA_DIR.mkdir(parents=True, exist_ok=True)
logging.info("BASE_DIR (make sure it's '*/heat-forecast/', else cd and re-run): %s", BASE_DIR)
logging.info("LOG_DIR: %s", LOG_DIR)
logging.info("OPTUNA_DIR: %s", OPTUNA_DIR)

Ensure [compatibility with Numba](https://numba.readthedocs.io/en/stable/user/installing.html#numba-support-info).

In [ ]:
# === Do not edit below ===
import sys, numpy, numba
logging.info("=== Current Environment ===")
logging.info("Python : %s", sys.version.split()[0])
logging.info("NumPy  : %s", numpy.__version__)
logging.info("Numba  : %s", numba.__version__)

Imports:

In [ ]:
# --- IPython Magic ---
%load_ext autoreload
%autoreload 2

# --- Standard Library ---
import sys
import yaml
import warnings
import logging
from collections import defaultdict
from datetime import datetime
from tqdm.notebook import tqdm
from typing import Any, Dict, Optional, Tuple, Iterable, Callable
from functools import partial

# --- Logging Configuration ---
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%H:%M:%S",
    force=True,
    stream=sys.stdout
)

# --- Data Handling ---
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.3f}'.format)

# --- Plotting & Visualization ---
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

# Set plot style and parameters
plt.style.use("seaborn-v0_8")
mpl.rcParams.update({
    'font.size': 12,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 14,
    'axes.grid': True,
    'axes.grid.which': 'both',
})

# --- Forecasting Models ---
from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive

# --- Project-Specific Utilities ---
from heat_forecast.utils.plotting import (
    plotly_cutoffs_with_exog, display_scrollable,
    add_season_background, plotly_daily_seasonality,
    plot_acf_diagnostics
)

from heat_forecast.utils.evaluation import (
    evaluate_cv_forecasts, display_cv_summary, cv_evaluation_summary,
    compute_loss_diffs, plotly_cv_metric_by_cutoff,
    by_horizon_preds, compute_error_stats_by_horizon,
    plotly_models_vs_horizon
)
from heat_forecast.utils.cv_utils import display_info_cv

logging.info("All imports successful.")

Import pre-elaborated data.

In [ ]:
heat_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'heat.csv'
aux_path = BASE_DIR / 'data' / 'timeseries_preprocessed' / 'auxiliary.csv'
heat_df = pd.read_csv(heat_path, parse_dates=['ds'])
aux_df = pd.read_csv(aux_path, parse_dates=['ds'])

## Load Saved Test Results

In [ ]:
TESTS_DIR = BASE_DIR / 'results' / 'test'
cv_groups = defaultdict(list)    # {(Fi, horizon): [(cv_df, run_dir)]}
run_tracker = defaultdict(list)  # {(model, Fi, horizon): [(timestamp, run_dir)]}
model_folder_to_name_map = {
    "mstl": "MSTL",
    "sarimax": "SARIMAX",
    "lstm": "LSTM",
    "xgboost": "XGBoost",
    "svmr": "SVMR",
    "tft": "TFT",
}
cutoffs = None  # Will be set to the unique cutoffs from the first loaded DataFrame

# --- Collect all valid run_dirs ---
for model_dir in TESTS_DIR.iterdir():
    if not model_dir.is_dir():
        continue

    for run_dir in model_dir.iterdir():
        if not run_dir.is_dir():
            logging.debug(f"Unexpected non-directory in folder: {run_dir.relative_to(TESTS_DIR)}. "
                            f"Non-directory: {run_dir}.")
            continue

        parts = run_dir.name.split('_')
        if len(parts) < 3 or not parts[0].startswith('F'):
            logging.debug(f"Skipping folder {run_dir.relative_to(TESTS_DIR)} due to unexpected format. "
                            f"Folder name: {run_dir.name}. "
                            f"Expected format: <unique_id>_<horizon_type>_test_<model_name>_<timestamp>.")
            continue

        fi = parts[0]
        horizon = parts[1]
        timestamp_str = parts[-1]
        model_run = parts[-2]
        if model_run not in model_folder_to_name_map.keys():
            if model_run.split('-')[0] in model_folder_to_name_map.keys(): # -> it's a variant of that model, ignore
                continue
            logging.debug(f"Model '{model_run}' (from run {run_dir.relative_to(TESTS_DIR)}) not recognized. "
                            f"Update the dict model_folder_to_name_map at the top of this cell."
                            f"Using model_run as model name.")
        if model_dir.name not in model_folder_to_name_map.keys():
            logging.debug(f"Model for folder '{model_dir.name}' not recognized. "
                            f"Update the dict model_folder_to_name_map at the top of this cell."
                            f"Using folder name as model name.")
        model = model_folder_to_name_map.get(model_dir.name, model_dir.name)

        try:
            timestamp = datetime.strptime(timestamp_str, "%Y%m%dT%H%M%S")
        except ValueError:
            logging.debug(f"Skipping run with invalid timestamp: {run_dir}")
            continue

        run_tracker[(model, fi, horizon)].append((timestamp, run_dir))

# ----------- Select latest runs and load them -----------
for key, runs in run_tracker.items():
    model, fi, horizon = key
    runs_sorted = sorted(runs, key=lambda x: x[0], reverse=True)

    if len(runs_sorted) > 1:
        logging.debug(f"Multiple runs found for model={model}, id={fi}, horizon={horizon}. Using latest only.")

    latest_run_dir = runs_sorted[0][1]
    cv_path = latest_run_dir / "cv_df.parquet"

    if not cv_path.exists():
        logging.debug(f"cv_df.parquet not found for latest run: {latest_run_dir}")
        continue

    try:
        df = pd.read_parquet(cv_path)
    except Exception as e:
        logging.debug(f"Failed to load {cv_path}: {e}")
        raise

    if not {'unique_id', 'ds'}.issubset(df.columns):
        raise ValueError(f"DataFrame from {cv_path} does not contain required columns ['unique_id', 'ds']")

    df.drop(columns=['y'], inplace=True, errors='ignore')  # Drop 'y' column if it exists
    df = df.set_index(['unique_id', 'ds', 'cutoff'])
    df.index.names = ['unique_id', 'ds', 'cutoff']
    cv_groups[(fi, horizon)].append((df, latest_run_dir, model))

# Build tidy records
rows = []
for (fi, horizon), values in cv_groups.items():
    for el in values:
        rows.append({
            "unique_id": fi,
            "horizon": horizon,
            "model": el[2],
            "run": str(el[1].relative_to(TESTS_DIR)),
        })

# Log it nicely
df = (
    pd.DataFrame(rows)
      .sort_values(["unique_id", "horizon", "model", "run"])
      .reset_index(drop=True)
)
df = df.set_index(["unique_id", "horizon", "model"])[["run"]]
with pd.option_context("display.max_rows", None, "display.width", 160):
    logging.info("Successfully selected the following runs:")
    display(df)

# ----------- Merge DataFrames -----------
merged_results_tmp = {} # {(Fi, horizon): (merged_df, for_cv)}
times = {}  # {(Fi, horizon): times_df}

for (fi, horizon), value in cv_groups.items():
    dfs, run_dirs, models = zip(*value)  

    rows = []           

    for i in range(len(dfs)):
        run_dir = run_dirs[i]
        model_name = models[i]
        df = dfs[i]

        # default record (in case metadata is missing)
        rec = {
            "model": model_name,
            "run_dir": str(run_dir),
            "step_size": pd.NA,
            "test_hours": pd.NA,
            "end_test_cv": pd.NA,
            "avg_el_per_fit": pd.NA,
        }

        metadata_path = run_dir / "metadata.yaml"
        try:
            with open(metadata_path, "r") as f:
                md = yaml.safe_load(f)
            if not isinstance(md, dict):
                md = {}
        except FileNotFoundError:
            logging.debug(f"Metadata file not found for {run_dir}.")
            rows.append(rec)
            continue

        for_cv = md.get("for_cv", {}) if isinstance(md.get("for_cv", {}), dict) else {}

        ss  = for_cv.get("step_size", pd.NA)
        th  = for_cv.get("test_hours", pd.NA)
        etc = for_cv.get("end_test_cv", pd.NA)
        ae  = md.get("avg_el_per_fit", pd.NA)
        ss  = pd.to_numeric(ss, errors="coerce") if ss is not pd.NA else pd.NA
        th  = pd.to_numeric(th, errors="coerce") if th is not pd.NA else pd.NA
        etc = pd.to_datetime(etc, errors="coerce") if etc is not pd.NA else pd.NA
        ae  = pd.to_numeric(ae, errors="coerce") if ae is not pd.NA else pd.NA

        if horizon == 'week' and ss is not pd.NA and ss == 24:
            logging.debug(f"Adjusting step_size from 24 to 168 for weekly horizon in {run_dir}.")
            ss = 24*7
            h = 24*7
            n_windows = (th - h) // ss + 1
            all_cutoffs = pd.to_datetime(df.reset_index()['cutoff'].unique())
            weekly_cutoffs = [
                etc - pd.Timedelta(hours=th) + pd.Timedelta(hours=ss * i)
                for i in range(n_windows)
            ]
            expected = all_cutoffs[::7]
            assert set(weekly_cutoffs) == set(expected), (
                f"Weekly cutoffs do not match expected for {run_dir}: \n"
                f"actual = \n{weekly_cutoffs}, \n"
                f"expected = \n{expected}, \n" 
                f"th_actual = {(weekly_cutoffs[-1] + pd.Timedelta(hours=h) - weekly_cutoffs[0]).total_seconds() // 3600} \n"
                f"th_expected = {(expected[-1] + pd.Timedelta(hours=h) - expected[0]).total_seconds() // 3600}"
            )
            etc_actual = weekly_cutoffs[-1] + pd.Timedelta(hours=h)
            th_actual = int((etc_actual - weekly_cutoffs[0]).total_seconds() // 3600)
            etc = etc_actual
            th = th_actual
            mask = df.index.get_level_values('cutoff').isin(weekly_cutoffs)
            df = df[mask]

        rec.update({
            "step_size": ss,
            "test_hours": th,
            "end_test_cv": etc,
            "avg_el_per_fit": ae,
        })

        to_log = []
        if pd.isna(ss) or pd.isna(th) or pd.isna(ae):
            if pd.isna(ss) or pd.isna(th):
                to_log.append("Missing CV params")
            if pd.isna(ae):
                to_log.append("Missing elapsed")
            logging.debug(f"{' & '.join(to_log)} for {run_dir}.")

        rows.append(rec)

        if i == 0:
            merged_df = df.copy()
        else:
            merged_df = merged_df.join(df, how='outer')

    merged_df = merged_df.reset_index()

    # Build the DataFrame you expect: one row per model/run
    for_cv_df = pd.DataFrame.from_records(rows, index='model')
    if len(for_cv_df['step_size'].dropna().unique()) > 1:
        raise ValueError(f"Different step sizes detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['step_size'].dropna().unique()}")
    if len(for_cv_df['test_hours'].dropna().unique()) > 1:
        raise ValueError(f"Different test hours detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['test_hours'].dropna().unique()}")
    if len(for_cv_df['end_test_cv'].dropna().unique()) > 1:
        raise ValueError(f"Different end_test_cv detected for Fi={fi}, horizon={horizon}:\n {for_cv_df['end_test_cv'].dropna().unique()}")

    # we assume at least one metadata was found
    for_cv = {
        'step_size': for_cv_df['step_size'].dropna().unique()[0],
        'test_hours': for_cv_df['test_hours'].dropna().unique()[0],
        'end_test_cv': for_cv_df['end_test_cv'].dropna().unique()[0],
    }

    merged_results_tmp[(fi, horizon)] = (merged_df, for_cv)
    times[(fi, horizon)] = for_cv_df["avg_el_per_fit"]

merged_results = {}  # {(Fi, horizon): merged_df}

# ----------- Add Naive forecast -----------
for (fi, horizon), (df, for_cv) in merged_results_tmp.items():
    naive_model = SeasonalNaive(season_length=24, alias='Naive24h')
    sf = StatsForecast(models=[naive_model], freq='h')

    heat_id_df = heat_df[heat_df['unique_id'] == fi].copy()
    
    h = 24*7 if horizon == 'week' else 24
    step_size = for_cv['step_size']
    test_hours = for_cv['test_hours']
    end_test_cv = for_cv['end_test_cv']
    n_windows = (test_hours - h) // step_size + 1
    full_df = heat_id_df[heat_id_df['ds'] <= end_test_cv]  

    cv_naive = sf.cross_validation(
        h=h,
        df=full_df,
        n_windows=n_windows,
        step_size=step_size,  
        test_size=test_hours,  
        input_size=None,  
        refit=True,  
    )

    df = df.merge(
        cv_naive,
        on=['unique_id', 'ds', 'cutoff'],
        how='outer',
    )

    merged_results[(fi, horizon)] = df.copy()

## Series F1

Plot target during test period:

In [ ]:
def plot_target_series(heat_df: pd.DataFrame, unique_id: str):
    y = heat_df[heat_df['unique_id'] == unique_id].set_index('ds')['y']
    y_test = y[y.index > pd.Timestamp('2024-09-01')]

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=y_test.index,
            y=y_test.values,
            mode="lines",
        )
    )
    fig.update_layout(
        title=f"Hourly target series - {unique_id}",
        xaxis_title="Time",
        yaxis_title="Heat demand (kW/h)",
        hovermode="x unified",
        template="plotly_white",
        width=1000,
        height=500,
    )

    # Add background fill by season
    add_season_background(fig, y_test.index.min(), y_test.index.max())

    html = fig.to_html(include_plotlyjs="inline", full_html=False)
    display(HTML(html))

plot_target_series(heat_df, 'F1')

Plot daily mean of target:

In [ ]:
def plot_target_series_daily_agg(heat_df: pd.DataFrame, unique_id: str):
    y = heat_df[heat_df['unique_id'] == unique_id].set_index('ds')['y']
    y_test = y[y.index > pd.Timestamp('2024-09-01')]
    y_test_daily = y_test.resample('D').mean()

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=y_test_daily.index,
            y=y_test_daily.values,
            mode="lines",
        )
    )
    fig.update_layout(
        title="Daily average of the target series",
        xaxis_title="Time",
        yaxis_title="Average heat demand (kW/h)",
        hovermode="x unified",
        template="plotly_white",
        width=1000,
        height=500,
    )

    # Add background fill by season
    add_season_background(fig, y_test.index.min(), y_test.index.max())

    html = fig.to_html(include_plotlyjs="inline", full_html=False)
    display(HTML(html))

plot_target_series_daily_agg(heat_df, 'F1')

### Daily horizon

In [ ]:
cv_df = merged_results[('F1', 'day')].copy()
times_df = times[('F1', 'day')].copy()
display_info_cv(cv_df)

#### All

In [ ]:
display_info_cv(cv_df)

##### Per-Window Error Statistics

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

##### Visualization of Per-Window Errors

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    width=3000,
    as_lineplot=True
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX', 'LD-TFT'],
    width=3000,
    as_lineplot=True
)
display_scrollable(fig) 

In [ ]:
_ = plot_acf_diagnostics(
    df=autumn_F1_day_df,
    cols_to_plot=['SARIMAX', 'XGBoost', 'LSTM'],
    acf_max_lag=200,
)

##### Visualization of Forecasts

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['MSTL', 'Naive24h'],
    highlight_dayofweek=True,
    #order_of_models=['SARIMAX', 'MSTL'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Autumn

In [ ]:
autumn_F1_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F1_day_df)

##### Per-Window Error Statistics

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

##### Visualization of Per-Window Errors

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
_ = plot_acf_diagnostics(
    df=autumn_F1_day_df,
    cols_to_plot=['SARIMAX', 'XGBoost', 'LSTM'],
    acf_max_lag=200,
)

##### Visualization of Forecasts

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['MSTL', 'Naive24h'],
    highlight_dayofweek=True,
    #order_of_models=['SARIMAX', 'MSTL'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F1_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
winter_target = heat_df[
    (heat_df['unique_id'] == 'F1') &
    (heat_df['ds'] >= pd.Timestamp('2024-12-21')) &
    (heat_df['ds'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F1_day_df)

##### Per-Window Error Statistics

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,#[['LSTM', 'TFT', 'unique_id', 'ds', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
ws = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='Naive24h'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='smape',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    aux_df=None,
    as_lineplot=False,
    height_per_row=400,
    grayscale_safe=False,
    width=2000
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-MSTL', 'LD-SARIMAX'],
    as_lineplot=False,
    grayscale_safe=False,
    width=1100
)
display_scrollable(fig) 

In [ ]:
from dieboldmariano import dm_test

L = all_results[all_results['metric'] == "mae"]['LSTM'].values
X = all_results[all_results['metric'] == "mae"]['XGBoost'].values
T = np.zeros(len(L))

dm_test(T, L, X, h=7)

In [ ]:
from scipy import stats

t_stat, p_val = stats.ttest_rel(L, X)  # equivalent
print(t_stat, p_val)

##### Analysis of per-hour-ahead errors

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F1_day_df)
per_h_summ, per_h_results = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    log_every=None,
    show_per_cutoff=False,
    evaluate_fn=partial(evaluate_cv_forecasts, metrics=['me']),
    summarize_fn=cv_evaluation_summary,
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)


In [ ]:
# Target grouped by month-dayhour
plotly_daily_seasonality(
    winter_target,
    show_legend=True,
    colors=px.colors.qualitative.Dark2
)

# Errors gouped by horizon
agg = 'mean'
_ = plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'TFT', 'SVMR'],
                         metric='me', 
                         with_bootstrap_band=True,
                         per_h_results=per_h_results,
                         agg=agg)
_ = plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                         models=['LSTM', 'XGBoost', 'SARIMAX', 'TFT', 'SVMR'],
                         with_bootstrap_band=True,
                         per_h_results=per_h_results,
                         metric='smape', agg=agg)

##### Analysis of Residuals

In [ ]:
payload, _ = plot_acf_diagnostics(
    winter_F1_day_df,
    cols_to_plot=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
residuals_F1 = pd.DataFrame({m: payload[m]['hist_x'] for m in payload.keys()})
corr_F1 = residuals_F1.corr()
display(corr_F1)

In [ ]:
import statsmodels.api as sm
import itertools
results = {}
for m1, m2 in itertools.combinations(residuals.columns, 2):
    X = sm.add_constant(residuals[m1])
    y = residuals[m2]
    model = sm.OLS(y, X).fit()
    results[(m1, m2)] = model.rsquared
models = residuals.columns
rsq_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in results.items():
    rsq_matrix.loc[m1, m2] = v
    rsq_matrix.loc[m2, m1] = v
rsq_matrix

In [ ]:
q = 0.80
indicator = residuals.abs().apply(lambda s: (s > s.quantile(q)).astype(int))
cooccurrence = {}
def jaccard(a, b):
    inter = ((a == 1) & (b == 1)).sum()
    union = ((a == 1) | (b == 1)).sum()
    return inter / union if union > 0 else np.nan
for m1, m2 in itertools.combinations(indicator.columns, 2):
    a, b = indicator[m1], indicator[m2]
    jacc = ((a == 1) & (b == 1)).sum() / ((a == 1) | (b == 1)).sum()
    cooccurrence[(m1, m2)] = jacc
models = residuals.columns
cooccurrence_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in cooccurrence.items():
    cooccurrence_matrix.loc[m1, m2] = v
    cooccurrence_matrix.loc[m2, m1] = v
cooccurrence_matrix

In [ ]:
q = 0.9
indicator = residuals.abs().apply(lambda s: (s > s.quantile(q)).astype(int))
indicator_with_magnitude = indicator * residuals.abs()
cooccurrence = {}

def jaccard_mod(a, b, a_mag=None, b_mag=None):
    inter = ((a == 1) & (b == 1))
    union = ((a == 1) | (b == 1)).sum()
    if a_mag is not None and b_mag is not None:
        inter = a_mag[inter].sum() + b_mag[inter].sum()
        union = a_mag[a == 1].sum() + b_mag[b == 1].sum()
    return inter / union if union > 0 else np.nan

for m1, m2 in itertools.combinations(indicator.columns, 2):
    a, b = indicator[m1], indicator[m2]
    a_mag = indicator_with_magnitude[m1]
    b_mag = indicator_with_magnitude[m2]
    #jacc = ((a == 1) & (b == 1)).sum() / ((a == 1) | (b == 1)).sum()
    jacc = jaccard_mod(a, b, a_mag, b_mag)
    cooccurrence[(m1, m2)] = jacc
models = residuals.columns
cooccurrence_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in cooccurrence.items():
    cooccurrence_matrix.loc[m1, m2] = v
    cooccurrence_matrix.loc[m2, m1] = v
cooccurrence_matrix

#### Visualization

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    models=['XGBoost', 'MSTL'],
    id='F1',
    highlight_dayofweek=True,
    alpha=1.0,
    base_height_per_panel=250,
    exog_vars=['temperature'],
    aux_df=aux_df,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F1_day_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F1_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F1_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    id='F1',
    highlight_dayofweek=False,
    order_of_models=['SARIMAX', 'LSTM', 'XGBoost'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
def compute_deltas(
    all_results: pd.DataFrame,
    stationarity_period: Tuple[pd.Timestamp, pd.Timestamp],
    metrics: Iterable[str],
    model_cols: Iterable[str],
    base_model: str = "LSTM",
    return_plot: bool = True,
):
    deltas_all_dict = {}
    deltas_stat_dict = {}
    for metric in metrics:
        # Select relevant models
        errors = all_results.loc[all_results['metric'] == metric, model_cols+['cutoff']].reset_index(drop=True)
        errors = errors[errors['cutoff']>pd.Timestamp('2024-08-10')].reset_index(drop=True)
        delta_cols = {
            col: errors[col] - errors[base_model]
            for col in model_cols
            if col != base_model
        }
        deltas_all = pd.DataFrame({
            'cutoff': errors['cutoff'],
            **delta_cols
        })
        deltas_all_dict[metric] = deltas_all
        # Select a period of approx stationarity
        deltas_stationary = deltas_all[
            deltas_all['cutoff'].between(stationarity_period[0], stationarity_period[1])
        ].reset_index(drop=True)
        deltas_stat_dict[metric] = deltas_stationary
    if return_plot:
        figs: list[go.Figure] = []
        for metric, deltas in deltas_all_dict.items():
            fig = go.Figure()
            delta_model_cols = set(model_cols).difference(['LSTM'])
            for col in delta_model_cols:
                fig.add_trace(
                    go.Scatter(
                        x=deltas['cutoff'],
                        y=deltas[col],
                        mode='lines',
                        name=col
                    )
                )
            first_x, sec_x = stationarity_period[0], stationarity_period[1]
            fig.add_shape(
                type="line",
                x0=first_x, x1=first_x, y0=0, y1=1,
                xref="x", yref="paper",
                line=dict(color="red", width=1, dash="dash"),
            )
            fig.add_shape(
                type="line",
                x0=sec_x, x1=sec_x, y0=0, y1=1,
                xref="x", yref="paper",
                line=dict(color="red", width=1, dash="dash"),
            )
            add_season_background(fig, deltas['cutoff'].min(), deltas['cutoff'].max())
            fig.update_layout(title=f"{metric.upper()} deltas vs {base_model} (n° cutoffs = {len(deltas_stat_dict[metric])})",)
            figs.append(fig)
        return deltas_stat_dict, figs
    return deltas_stat_dict, []

deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
import statsmodels.api as sm

metrics = list(deltas_dict.keys())
delta_model_cols = deltas_dict[metrics[0]].columns.difference(['cutoff']).tolist()
base_model = 'LSTM'
model_cols = delta_model_cols + [base_model]

max_lag = 40  

acf_figs = []

for metric in metrics:
    deltas = deltas_dict[metric]
    for col in delta_model_cols:
        series = deltas[col].dropna().to_numpy()
        acf_vals = sm.tsa.stattools.acf(series, nlags=max_lag, fft=True)

        # approximate 95 % confidence band for white noise
        n = len(series)
        conf = 1.96 / np.sqrt(n)

        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                x=list(range(len(acf_vals))),
                y=acf_vals,
                name=f"ACF {metric.upper()} – {col} vs {base_model}"
            )
        )
        fig.add_hline(y=conf, line_width=1, line_dash="dash")
        fig.add_hline(y=-conf, line_width=1, line_dash="dash")

        fig.update_layout(
            title=f"ACF of stationary deltas – {metric.upper()} – {col} vs {base_model}",
            xaxis_title="Lag",
            yaxis_title="Autocorrelation"
        )
        acf_figs.append(fig)

# display all ACF figures (in Jupyter)
for fig in acf_figs:
    display(fig)


In [ ]:
from plotly.subplots import make_subplots
from recombinator.optimal_block_length import optimal_block_length
from recombinator.block_bootstrap import circular_block_bootstrap

def bootstrap_mean_delta(
    deltas_dict: Dict[str, pd.DataFrame],
    n_boot: int = 10_000,
    alpha: float = 0.05,
    n_cols: int = 2,
    width_per_col: int = 400,
    height_per_row: int = 320,
    block_length: Optional[int] = None,
) -> Tuple[Dict[str, pd.DataFrame], go.Figure]:
    results: Dict[str, pd.DataFrame] = {}

    # ---------- compute bootstrap stats per metric ----------
    for metric, deltas in deltas_dict.items():
        delta_cols = deltas.columns.difference(["cutoff"])
        results_metric = []

        for col in delta_cols:
            x = deltas[col].to_numpy()
            x = x[~np.isnan(x)]

            # Optimal block length for this series
            if block_length is not None:
                b_cb = block_length
            else:
                b_star = optimal_block_length(x)[0]
                b_cb = int(np.ceil(b_star.b_star_cb))

            # Circular block bootstrap
            samples = circular_block_bootstrap(
                x,
                block_length=b_cb,
                replications=n_boot,
                replace=True,
            )  # shape = (replications, len(x))

            boot_means = samples.mean(axis=1)

            # CI (percentile bootstrap)
            q_low, q_high = np.percentile(
                boot_means,
                [100 * alpha / 2, 100 * (1 - alpha / 2)],
            )

            # Two-sided p-value for H0: Delta = 0 vs H1: Delta != 0
            obs = x.mean()  # media dei valori osservati
            null_samples = boot_means - obs # boot_means sono ottenuti pescando con ripetizione dai valori osservati,
                                            # traslando stiamo "simulando la distribuzione nulla"
            p_two_sided = np.mean(np.abs(null_samples) >= np.abs(obs))

            results_metric.append(
                {
                    "model": col,
                    "mean_delta": x.mean(),
                    "ci_low": q_low,
                    "ci_high": q_high,
                    "block_length_cb": b_cb,
                    "p_raw": p_two_sided,
                }
            )

        # Collect everything in a DataFrame
        ci_df = pd.DataFrame(results_metric)

        # ---- Holm–Bonferroni correction (within this metric) ----
        p = ci_df["p_raw"].values
        m = len(p)
        order = np.argsort(p)
        p_sorted = p[order]

        # Step-down Holm adjustment
        q = (m - np.arange(m)) * p_sorted  # (m-k+1) * p_(k)
        adj_sorted = np.empty(m)
        running_max = 0.0
        for i in range(m):
            val = min(q[i], 1.0)
            running_max = max(running_max, val)
            adj_sorted[i] = running_max

        p_holm = np.empty(m)
        p_holm[order] = adj_sorted  # unsort back

        ci_df["p_holm"] = p_holm

        results[metric] = ci_df

    # ---------- forest plots in subplots ----------
    metrics = list(results.keys())
    n_metrics = len(metrics)
    n_rows = int(np.ceil(n_metrics / n_cols))

    fig = make_subplots(
        rows=n_rows,
        cols=n_cols,
        subplot_titles=[m.upper() for m in metrics],
        shared_yaxes=False,
    )

    for idx, metric in enumerate(metrics):
        row = idx // n_cols + 1
        col = idx % n_cols + 1

        ci_df = results[metric]
        plot_df = ci_df.sort_values("mean_delta").reset_index(drop=True)

        # Mean Δ points
        fig.add_trace(
            go.Scatter(
                x=plot_df["model"],
                y=plot_df["mean_delta"],
                mode="markers",
                marker=dict(size=8, color="black"),
                name="Mean Δ",
                showlegend=(idx == 0),
            ),
            row=row,
            col=col,
        )

        # CI bars (lower bound)
        fig.add_trace(
            go.Scatter(
                x=plot_df["model"],
                y=plot_df["ci_low"],
                mode="lines",
                line=dict(width=1, color="gray"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

        # CI bars (upper bound) + fill between
        fig.add_trace(
            go.Scatter(
                x=plot_df["model"],
                y=plot_df["ci_high"],
                mode="lines",
                line=dict(width=1, color="gray"),
                fill="tonexty",
                fillcolor="rgba(100,100,100,0.15)",
                name=f"{int((1-alpha)*100)}% CI" if idx == 0 else None,
                showlegend=(idx == 0),
            ),
            row=row,
            col=col,
        )

        # Reference line: "no improvement" (Delta = 0)
        fig.add_hline(
            y=0,
            line_width=1,
            line_dash="dash",
            row=row,
            col=col,
        )

    fig.update_layout(
        title="Mean Δ with Bootstrap 95% CI",
        xaxis_title="Model",
        yaxis_title="Mean Δ",
        template="simple_white",
        width=width_per_col * n_cols,
        height=height_per_row * n_rows,
    )

    return results, fig

results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F1', 'week')]
times_df = times[('F1', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F1_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    width=1000,
)
display_scrollable(fig)    

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-TFT'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
cutoffs_all = autumn_F1_week_df['cutoff'].sort_values().unique()

fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F1_week_df[autumn_F1_week_df['cutoff'].isin(cutoffs_all)],
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F1_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df)

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-TFT'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F1_week_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F1_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-TFT'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F1_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'TFT'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR', 'TFT'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

## Series F2

In [ ]:
plot_target_series(heat_df, 'F2')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F2')

### Daily horizon

In [ ]:
cv_df = merged_results[('F2', 'day')]
times_df = times[('F2', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F2_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F2_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='smape',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)   

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F2_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
winter_target = heat_df[
    (heat_df['unique_id'] == 'F2') &
    (heat_df['ds'] >= pd.Timestamp('2024-12-21')) &
    (heat_df['ds'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F2_day_df)

In [ ]:
corrupted_cutoffs = [
    pd.Timestamp('2025-01-20T23:00:00'),
    pd.Timestamp('2025-01-21T23:00:00'),
    pd.Timestamp('2025-01-22T23:00:00'),
]
all_results = evaluate_cv_forecasts(
    cv_df=winter_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20')),
)
summary = cv_evaluation_summary(
    all_results,
    ignore_cutoffs=corrupted_cutoffs
)

_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost',
)
ld_summary = cv_evaluation_summary(
    all_loss_diffs,
    ignore_cutoffs=corrupted_cutoffs
)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

##### Analysis of per-hour-ahead errors

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F2_day_df)
per_h_summ = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    step=1,
    log_every=5,
    show_per_cutoff=True,
    evaluate_fn=evaluate_cv_forecasts,
    summarize_fn=partial(
        cv_evaluation_summary,
        ignore_cutoffs=corrupted_cutoffs
    ),
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)


In [ ]:
# Target grouped by month-dayhour
plotly_daily_seasonality(
    winter_target,
    show_legend=True
)

# Errors gouped by horizon
agg = 'mean'
plotly_models_vs_horizon(per_h_summ=per_h_summ, metric='mae', agg=agg)
plotly_models_vs_horizon(per_h_summ=per_h_summ, metric='smape', agg=agg)

##### Analysis of Residuals

In [ ]:
payload, _ = plot_acf_diagnostics(
    winter_F2_day_df[~winter_F2_day_df['cutoff'].isin(corrupted_cutoffs)],
    cols_to_plot=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
residuals_F2 = pd.DataFrame({m: payload[m]['hist_x'] for m in payload.keys()})
corr_F2 = residuals_F2.corr()
display(corr_F2)

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F2_day_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F2_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F2_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)    

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F2_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F2', 'week')]
times_df = times[('F2', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F2_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F2_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F2_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F2_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F2_week_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F2_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F2_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df)  

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F2_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

## Series F3

In [ ]:
plot_target_series(heat_df, 'F3')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F3')

### Daily horizon

In [ ]:
cv_df = merged_results[('F3', 'day')]
times_df = times[('F3', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F3_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F3_day_df,#[['XGBoost', 'SVMR', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'SVMR'],
    width=2000,
)
display_scrollable(fig)   

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F3_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F3_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX', 'LD-SVMR'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

##### Analysis of Residuals

In [ ]:
data, _ = plot_acf_diagnostics(
    winter_F3_day_df,
    cols_to_plot=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
residuals_F3 = pd.DataFrame({m: data[m]['hist_x'] for m in data.keys()})
corr_F3 = residuals_F3.corr()
display(corr_F3)

In [ ]:
import statsmodels.api as sm
import itertools
results = {}
for m1, m2 in itertools.combinations(residuals.columns, 2):
    X = sm.add_constant(residuals[m1])
    y = residuals[m2]
    model = sm.OLS(y, X).fit()
    results[(m1, m2)] = model.rsquared
models = residuals.columns
rsq_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in results.items():
    rsq_matrix.loc[m1, m2] = v
    rsq_matrix.loc[m2, m1] = v
rsq_matrix


In [ ]:
q = 0.9
indicator = residuals.abs().apply(lambda s: (s > s.quantile(q)).astype(int))
indicator_with_magnitude = indicator * residuals.abs()
cooccurrence = {}

def jaccard_mod(a, b, a_mag=None, b_mag=None):
    inter = ((a == 1) & (b == 1))
    union = ((a == 1) | (b == 1)).sum()
    if a_mag is not None and b_mag is not None:
        inter = a_mag[inter].sum() + b_mag[inter].sum()
        union = a_mag[a == 1].sum() + b_mag[b == 1].sum()
    return inter / union if union > 0 else np.nan

for m1, m2 in itertools.combinations(indicator.columns, 2):
    a, b = indicator[m1], indicator[m2]
    a_mag = indicator_with_magnitude[m1]
    b_mag = indicator_with_magnitude[m2]
    #jacc = ((a == 1) & (b == 1)).sum() / ((a == 1) | (b == 1)).sum()
    jacc = jaccard_mod(a, b, a_mag, b_mag)
    cooccurrence[(m1, m2)] = jacc
models = residuals.columns
cooccurrence_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in cooccurrence.items():
    cooccurrence_matrix.loc[m1, m2] = v
    cooccurrence_matrix.loc[m2, m1] = v
cooccurrence_matrix

##### Visualization

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F3_day_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F3_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F3_day_df,#[['LSTM', 'SVMR', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'SVMR'],
    width=2000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F3_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'LSTM', 'SARIMAX'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'LSTM', 'SARIMAX'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F3', 'week')]
times_df = times[('F3', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F3_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F3_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F3_week_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F3_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F3_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df)  

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'MSTL', 'SVMR'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-MSTL', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F3_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'MSTL', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

## Series F4

In [ ]:
plot_target_series(heat_df, 'F4')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F4')

### Daily horizon

In [ ]:
cv_df = merged_results[('F4', 'day')]
times_df = times[('F4', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F4_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F4_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-SARIMAX'],

)
display_scrollable(fig) 

##### Analysis of Residuals

In [ ]:
data, _ = plot_acf_diagnostics(
    winter_F4_day_df,
    cols_to_plot=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
residuals_F4 = pd.DataFrame({m: data[m]['hist_x'] for m in data.keys()})
corr_F4 = residuals_F4.corr()
display(corr_F4)

In [ ]:
import statsmodels.api as sm
import itertools
results = {}
for m1, m2 in itertools.combinations(residuals.columns, 2):
    X = sm.add_constant(residuals[m1])
    y = residuals[m2]
    model = sm.OLS(y, X).fit()
    results[(m1, m2)] = model.rsquared
models = residuals.columns
rsq_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in results.items():
    rsq_matrix.loc[m1, m2] = v
    rsq_matrix.loc[m2, m1] = v
rsq_matrix


In [ ]:
q = 0.9
indicator = residuals.abs().apply(lambda s: (s > s.quantile(q)).astype(int))
indicator_with_magnitude = indicator * residuals.abs()
cooccurrence = {}

def jaccard_mod(a, b, a_mag=None, b_mag=None):
    inter = ((a == 1) & (b == 1))
    union = ((a == 1) | (b == 1)).sum()
    if a_mag is not None and b_mag is not None:
        inter = a_mag[inter].sum() + b_mag[inter].sum()
        union = a_mag[a == 1].sum() + b_mag[b == 1].sum()
    return inter / union if union > 0 else np.nan

for m1, m2 in itertools.combinations(indicator.columns, 2):
    a, b = indicator[m1], indicator[m2]
    a_mag = indicator_with_magnitude[m1]
    b_mag = indicator_with_magnitude[m2]
    #jacc = ((a == 1) & (b == 1)).sum() / ((a == 1) | (b == 1)).sum()
    jacc = jaccard_mod(a, b, a_mag, b_mag)
    cooccurrence[(m1, m2)] = jacc
models = residuals.columns
cooccurrence_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in cooccurrence.items():
    cooccurrence_matrix.loc[m1, m2] = v
    cooccurrence_matrix.loc[m2, m1] = v
cooccurrence_matrix

#### Visualization

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F4_day_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F4_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F4_day_df,#[['LSTM', 'XGBoost', 'ds', 'unique_id', 'cutoff', 'y']],
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SVMR', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F4_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F4', 'week')]
times_df = times[('F4', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F4_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F4_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SARIMAX'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics 
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F4_week_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F4_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F4_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F4_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

## Series F5

In [ ]:
plot_target_series(heat_df, 'F5')

In [ ]:
plot_target_series_daily_agg(heat_df, 'F5')

### Daily horizon

In [ ]:
cv_df = merged_results[('F5', 'day')]
times_df = times[('F5', 'day')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F5_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR','SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F5_day_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['XGBoost', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

##### Analysis of Residuals

In [ ]:
data, _ = plot_acf_diagnostics(
    winter_F5_day_df,
    cols_to_plot=['SARIMAX','XGBoost','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
)

In [ ]:
residuals_F5 = pd.DataFrame({m: data[m]['hist_x'] for m in data.keys()})
corr_F5 = residuals_F5.corr()
display(corr_F5)

In [ ]:
import statsmodels.api as sm
import itertools
results = {}
for m1, m2 in itertools.combinations(residuals.columns, 2):
    X = sm.add_constant(residuals[m1])
    y = residuals[m2]
    model = sm.OLS(y, X).fit()
    results[(m1, m2)] = model.rsquared
models = residuals.columns
rsq_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in results.items():
    rsq_matrix.loc[m1, m2] = v
    rsq_matrix.loc[m2, m1] = v
rsq_matrix


In [ ]:
q = 0.9
indicator = residuals.abs().apply(lambda s: (s > s.quantile(q)).astype(int))
indicator_with_magnitude = indicator * residuals.abs()
cooccurrence = {}

def jaccard_mod(a, b, a_mag=None, b_mag=None):
    inter = ((a == 1) & (b == 1))
    union = ((a == 1) | (b == 1)).sum()
    if a_mag is not None and b_mag is not None:
        inter = a_mag[inter].sum() + b_mag[inter].sum()
        union = a_mag[a == 1].sum() + b_mag[b == 1].sum()
    return inter / union if union > 0 else np.nan

for m1, m2 in itertools.combinations(indicator.columns, 2):
    a, b = indicator[m1], indicator[m2]
    a_mag = indicator_with_magnitude[m1]
    b_mag = indicator_with_magnitude[m2]
    #jacc = ((a == 1) & (b == 1)).sum() / ((a == 1) | (b == 1)).sum()
    jacc = jaccard_mod(a, b, a_mag, b_mag)
    cooccurrence[(m1, m2)] = jacc
models = residuals.columns
cooccurrence_matrix = pd.DataFrame(np.zeros((len(models), len(models))),
                          index=models, columns=models)
for (m1, m2), v in cooccurrence.items():
    cooccurrence_matrix.loc[m1, m2] = v
    cooccurrence_matrix.loc[m2, m1] = v
cooccurrence_matrix

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['XGBoost', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['XGBoost', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F5_day_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F5_day_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F5_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-06-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='SVMR'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SVMR', 'SARIMAX', 'LSTM'],

)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX'],

)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F5_day_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SVMR', 'SARIMAX', 'LSTM'],
    highlight_dayofweek=False,
    order_of_models=['SVMR', 'SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

### Weekly horizon

In [ ]:
cv_df = merged_results[('F5', 'week')]
times_df = times[('F5', 'week')]
display_info_cv(cv_df)

#### Autumn

In [ ]:
autumn_F5_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-09-20')) &
    (cv_df['cutoff'] < pd.Timestamp('2024-12-21'))
].copy()
display_info_cv(autumn_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=autumn_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Winter

In [ ]:
winter_F5_week_df = cv_df[
    (cv_df['cutoff'] >= pd.Timestamp('2024-12-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-03-21'))
].copy()
display_info_cv(winter_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-12-21'), pd.Timestamp('2025-03-20'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-LSTM', 'LD-SARIMAX', 'LD-SVMR'],
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    highlight_dayofweek=False,
    order_of_models=['SARIMAX', 'LSTM'],
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Spring

In [ ]:
spring_F5_week_df = cv_df[
    (cv_df['cutoff'] > pd.Timestamp('2025-03-21')) &
    (cv_df['cutoff'] <= pd.Timestamp('2025-06-21'))
].copy()
display_info_cv(spring_F5_week_df)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=spring_F5_week_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2025-03-21'), pd.Timestamp('2025-05-12'))
)
summary = cv_evaluation_summary(all_results)
_ = display_cv_summary(summary, sort_metric='mae', times_df=times_df) 

In [ ]:
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='XGBoost'
)
ld_summary = cv_evaluation_summary(all_loss_diffs)
_ = display_cv_summary(ld_summary, are_loss_diffs=True)

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig)  

In [ ]:
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    width=1000,
)
display_scrollable(fig) 

In [ ]:
fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=spring_F5_week_df,
    start_offset=24*7,
    end_offset=24*3,
    aux_df=aux_df,
    exog_vars=['temperature'],
    models=['SARIMAX', 'LSTM', 'XGBoost'],
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=250,
)
display_scrollable(fig)

#### Block Bootstrap

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
for fig in deltas_figs:
    display(fig)

In [ ]:
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
display(fig)
display(df)

## Thesis Figures

In [ ]:
from heat_forecast.utils.plotting import extract_subplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
def set_plotly_style(fig):
    fig.update_layout(
        font=dict(size=12),
        title_font=dict(size=14),
        legend=dict(font=dict(size=12)),
    )
    fig.update_xaxes(tickfont=dict(size=12), title_font=dict(size=12))
    fig.update_yaxes(tickfont=dict(size=12), title_font=dict(size=12))
    return fig
def gray_shade(intensity: float):
    c = np.clip(1-intensity, 0, 1)
    return f"rgb({int(c*255)},{int(c*255)},{int(c*255)})"

In [ ]:
cutoffs = winter_F1_day_df['cutoff'].unique()
selected_cutoffs = cutoffs[(cutoffs > pd.Timestamp('2025-02-15 00:00:00')) & (cutoffs < pd.Timestamp('2025-03-02 00:00:00'))]
winter_F1_day_df_selected = winter_F1_day_df[winter_F1_day_df['cutoff'].isin(selected_cutoffs)]

fig = plotly_cutoffs_with_exog(
    target_df=heat_df,
    cv_df=winter_F1_day_df_selected,
    start_offset=0,
    end_offset=0,
    models=['MSTL'],
    id='F1',
    highlight_dayofweek=False,
    alpha=1.0,
    base_height_per_panel=200,
    width_per_day=45,
    exog_vars=['temperature'],
    aux_df=aux_df,
    grayscale_safe=True,
    title=None
)
fig = set_plotly_style(fig)
for r in range(1, 3):
    fig.update_xaxes(showgrid=True, tickformat=r"%b %d", row=r, col=1)
print(f"Figure width: {fig.layout.width} px")
print(f"Figure height: {fig.layout.height} px")
# import plotly.io as pio
# pio.write_html(
#     fig, "fig.html",
#     include_plotlyjs="inline", full_html=True,
#     config={"toImageButtonOptions": {"format": "svg"}}
# )
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig

In [ ]:
cutoffs_str

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=autumn_F1_day_df,
    target_df=heat_df, 
)
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
fig1 = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-TFT', 'LD-SARIMAX'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=False,
    width=650
)
cutoffs_str = all_loss_diffs['cutoff'].dt.strftime('%Y-%m-%d')
x_min, x_max = cutoffs_str.min(), cutoffs_str.max()
line_trace = go.Scatter(
    x=[x_min, x_max],
    y=[0, 0],
    mode="lines",
    line=dict(color="black", dash="dash"),
    showlegend=False
)
fig1.add_trace(line_trace)
fig1.data = (fig1.data[-1],) + fig1.data[:-1]
fig1.data[1].line.color = gray_shade(0.4)
fig1.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
    template="plotly_white",
)
for i in range(len(fig1.layout.annotations)):
    fig1.layout.annotations[i].text = ""
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
fig2 = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-TFT', 'LD-SARIMAX'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=False,
    width=650
)
cutoffs_str = all_loss_diffs['cutoff'].dt.strftime('%Y-%m-%d')
x_min, x_max = cutoffs_str.min(), cutoffs_str.max()
line_trace = go.Scatter(
    x=[x_min, x_max],
    y=[0, 0],
    mode="lines",
    line=dict(color="black", dash="dash"),
    showlegend=False
)
fig2.add_trace(line_trace)
fig2.data = (fig2.data[-1],) + fig2.data[:-1]
fig2.data[1].line.color = gray_shade(0.4)
fig2.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
    template="plotly_white",
)
for i in range(len(fig2.layout.annotations)):
    fig2.layout.annotations[i].text = ""
all_results = evaluate_cv_forecasts(
    cv_df=spring_F1_day_df,
    target_df=heat_df, 
)
all_loss_diffs = compute_loss_diffs(
    all_results,
    baseline_model='LSTM'
)
fig3 = plotly_cv_metric_by_cutoff(
    combined_results=all_loss_diffs,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LD-XGBoost', 'LD-TFT', 'LD-SARIMAX'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=False,
    width=650
)
cutoffs_str = all_loss_diffs['cutoff'].dt.strftime('%Y-%m-%d')
x_min, x_max = cutoffs_str.min(), cutoffs_str.max()
line_trace = go.Scatter(
    x=[x_min, x_max],
    y=[0, 0],
    mode="lines",
    line=dict(color="black", dash="dash"),
    showlegend=False
)
fig3.add_trace(line_trace)
fig3.data = (fig3.data[-1],) + fig3.data[:-1]
fig3.data[1].line.color = gray_shade(0.4)
fig3.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
    template="plotly_white",
)
for i in range(len(fig3.layout.annotations)):
    fig3.layout.annotations[i].text = ""

combined_fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=False,
    shared_yaxes=True,
    vertical_spacing=0.08,
    subplot_titles=("Autumn", "Winter", "Spring"),
     
)
for tr in fig1.data:
    combined_fig.add_trace(tr, row=1, col=1)
for tr in fig2.data:
    combined_fig.add_trace(tr, row=2, col=1)
for tr in fig3.data:
    combined_fig.add_trace(tr, row=3, col=1)
for tr in combined_fig.data[len(fig1.data):]:
    tr.showlegend = False
combined_fig.update_layout(
    legend=dict(
        x=0.81,
        y=0.15,  
        bgcolor="rgba(255,255,255,1)",
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    template="plotly_white",
    margin=dict(t=50),
    height=700,     
    width=750,
)
all_y_values = []
for fig in [fig1, fig2, fig3]:
    for trace in fig.data:
        if hasattr(trace, 'y') and trace.y is not None:
            all_y_values.extend(trace.y)
y_min = min(all_y_values)
y_max = max(all_y_values)
y_range = [y_min - 0.05 * (y_max - y_min), y_max + 0.05 * (y_max - y_min)]
combined_fig.update_yaxes(title_text=None, row=1, col=1, linecolor="lightgrey", mirror=True, range=y_range)
combined_fig.update_yaxes(title_text="MAE Difference vs LSTM", row=2, col=1, linecolor="lightgrey", mirror=True, range=y_range)
combined_fig.update_yaxes(title_text=None, row=3, col=1, linecolor="lightgrey", mirror=True, range=y_range)
combined_fig.update_xaxes(title_text=None, row=1, col=1, linecolor="lightgrey", mirror=True)
combined_fig.update_xaxes(title_text=None, row=2, col=1, linecolor="lightgrey", mirror=True)
combined_fig.update_xaxes(title_text="Cutoff", row=3, col=1, linecolor="lightgrey", mirror=True)

# Save and display
pio.write_html(
    combined_fig, "fig_merged.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(combined_fig)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LSTM', 'XGBoost', 'SARIMAX'],
    aux_df=None,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=True,
    width=650
)
fig.data = fig.data[:1] + fig.data[2:]
fig.data[1].line.color = gray_shade(0.4)
fig.update_layout(
    legend=dict(
        x=0.8,          
        y=1.0,          
        bgcolor="rgba(255,255,255,1)",  # legend background
        bordercolor="white",
        borderwidth=1,
        title=""
    ),
    title_text=None,
    margin=dict(t=20),
)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = ""
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=winter_F1_day_df,
    target_df=heat_df, 
    period_for_nmae=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21'))
)
fig = plotly_cv_metric_by_cutoff(
    combined_results=all_results,
    metric='mae',  # Change to 'rmse' or 'me' for other metrics
    models=['LSTM', 'XGBoost'],
    aux_df=aux_df,
    as_lineplot=True,
    height_per_row=300,
    grayscale_safe=True,
    width=650
)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].text = ""
fig = extract_subplot(fig, 2, 1)
fig.update_layout(
    height=300,
    width=650,
    margin=dict(l=60, r=20, t=20, b=60),
    showlegend=False
)
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig


In [ ]:
all_maes = all_results[all_results['metric'] == 'mae']
corr_LSTM_XGB = pd.Series.corr(all_maes['LSTM'], all_maes['XGBoost'])
print("Correlation between LSTM and XGBoost MAE errors:", corr_LSTM_XGB)
corr_LSTM_SARIMAX = pd.Series.corr(all_maes['LSTM'], all_maes['SARIMAX'])
print("Correlation between LSTM and SARIMAX MAE errors:", corr_LSTM_SARIMAX)
corr_SARIMAX_XGB = pd.Series.corr(all_maes['SARIMAX'], all_maes['XGBoost'])
print("Correlation between SARIMAX and XGBoost MAE errors:", corr_SARIMAX_XGB)

In [ ]:
by_horizon_dict = by_horizon_preds(winter_F1_day_df)
per_h_summ, per_h_res = compute_error_stats_by_horizon(
    by_horizon_dict=by_horizon_dict,
    target_df=heat_df,
    nmae_period=(pd.Timestamp('2024-09-20'), pd.Timestamp('2024-12-21')),
    log_every=None,
    show_per_cutoff=True,
    evaluate_fn=partial(evaluate_cv_forecasts, metrics=['mae', 'smape', 'me']),
    summarize_fn=cv_evaluation_summary,
    by_cutoff_plot_fn=partial(
        plotly_cv_metric_by_cutoff, 
        models=['LSTM', 'XGBoost', 'SARIMAX', 'SVMR'],
        width=2000
    ),
)

In [ ]:
# Target grouped by month-dayhour
fig1 = plotly_daily_seasonality(
    winter_target,
    show_legend=True,
    colors=px.colors.qualitative.Dark2,
    display_fig=False,
    width=750,
    height_per_id=350
)
for i in range(len(fig1.layout.annotations)):
    fig1.layout.annotations[i].text = ""
fig1.update_layout(
    margin=dict(l=60, r=20, t=20, b=60),
    title_text=None,
    legend=dict(
        x=1.0,
        y=1.0,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.7)",
        orientation="h",
    )
)
fig1.update_xaxes(range=[0, 23], mirror=True, linecolor="lightgrey", tickvals=np.arange(0,24))
fig1.update_yaxes(mirror=True, linecolor="lightgrey")
display(fig1) 
# Errors gouped by horizon
agg = 'mean'
metrics = ['mae', 'smape', 'me']
figs = []
for i in range(2):
    fig2 = plotly_models_vs_horizon(per_h_summ=per_h_summ, 
                            models=(
                                ['LSTM', 'XGBoost', 'SARIMAX', 'TFT', 'SVMR', 'MSTL', 'Naive24h'] if i < 2 
                                else ['LSTM', 'XGBoost', 'SARIMAX', 'TFT']
                            ),
                            metric=metrics[i], 
                            with_bootstrap_band=True,
                            per_h_results=per_h_res,
                            agg=agg,
                            display_fig=False,
                            width=650, height=320)
    for i in range(len(fig2.layout.annotations)):
        fig2.layout.annotations[i].text = ""
    fig2.update_layout(
        margin=dict(l=60, r=20, t=20, b=60),
        title_text=None,
    )
    fig2.update_xaxes(range=[1, 24])
    figs.append(fig2)
merged = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    vertical_spacing=0.05,
    row_heights=[0.5, 0.5],
)
for tr in figs[0].data:
    merged.add_trace(tr, row=1, col=1)
for tr in figs[1].data:
    merged.add_trace(tr, row=2, col=1)
#for tr in figs[2].data:
#    merged.add_trace(tr, row=3, col=1)
merged.update_yaxes(title_text="MAE", row=1, col=1, mirror=True, linecolor="lightgrey")
merged.update_yaxes(title_text="sMAPE", row=2, col=1, mirror=True, linecolor="lightgrey")
#merged.update_yaxes(title_text="ME", row=3, col=1, mirror=True, linecolor="lightgrey")
merged.update_xaxes(title_text="", row=1, col=1, mirror=True, linecolor="lightgrey", range=[1,24], tickvals=np.arange(1,25,1))
#merged.update_xaxes(title_text="", row=2, col=1, mirror=True, linecolor="lightgrey", range=[1,24], tickvals=np.arange(1,25,1))
merged.update_xaxes(title_text="Forecast Horizon (hours)", row=2, col=1, mirror=True, linecolor="lightgrey", range=[1,24], tickvals=np.arange(1,25,1))
merged.update_layout(
    height=750,
    width=750,
    margin=dict(l=60, r=20, t=30, b=60),
    showlegend=True, 
    template="plotly_white", 
    legend=dict(
        x=1.0,
        y=1.0,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.7)",
        orientation="h",
    )
)
for tr in merged.data[:len(fig2.data)]:
    if tr.name=="MSTL":
        tr.name="MSTL-ETS"
for tr in merged.data[len(fig2.data):]:
    tr.showlegend = False
pio.write_html(
    merged, "merged.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(merged)

In [ ]:
all_results = evaluate_cv_forecasts(
    cv_df=cv_df,
    target_df=heat_df, 
)
all_results.head()

In [ ]:
deltas_dict, deltas_figs = compute_deltas(
    all_results=all_results,
    stationarity_period=(pd.Timestamp('2024-11-10'), pd.Timestamp('2025-03-17')),
    metrics=['mae', 'rmse', 'smape'],
    model_cols=['SARIMAX', 'LSTM', 'XGBoost', 'SVMR'],
    base_model='LSTM',
    return_plot=True
)
results, fig = bootstrap_mean_delta(
    deltas_dict=deltas_dict,
    n_boot=10_000,
    alpha=0.05,
    n_cols=3,
    width_per_col=400,
    height_per_row=320,
)
df = pd.concat(
    {metric: df.set_index("model") for metric, df in results.items()},
    axis=0
)
for i in range(3):
    fig.update_yaxes(row=1, col=i+1, mirror=True, linecolor="lightgrey")
    fig.update_xaxes(row=1, col=i+1, mirror=True, linecolor="lightgrey")
fig.update_layout(
    title_text=None,
    margin=dict(l=60, r=20, t=30, b=60),
    xaxis_title=None,
    showlegend=False,
    width=750,
    height=350,
    template="plotly_white",
)
fig.update_xaxes(title_text="Model", row=1, col=2)
fig.layout.annotations[2].update(text="sMAPE")
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
display(fig)
display(df)

In [ ]:
payload, fig = plot_acf_diagnostics(
    winter_F1_day_df,
    cols_to_plot=['TFT', 'XGBoost', 'SARIMAX','LSTM'],
    acf_max_lag=168,  # e.g., up to weekly lags for hourly data
    num_bins=50
)
for m in payload.keys():
    bias = payload[m]['hist_x'].mean()
    print(f"bias for {m}: {bias:.4f}")

In [ ]:
import plotly.graph_objects as go
from heat_forecast.utils.plotting import extract_subplot

subfig = extract_subplot(fig, row=2, col=1)
subfig.update_layout(
    barmode="overlay",
    template="plotly_white",
    margin=dict(l=60, r=20, t=20, b=30),
    legend=dict(
        x=1.0,
        y=1.0,
        bgcolor="rgba(255,255,255,1)",  # legend background
        xanchor="right",
        yanchor="top",
    ),
    height=400,
    width=750,
)
rename = {
    "LSTM ACF": "LSTM",
    "SARIMAX ACF": "SARIMAX",
    "XGBoost ACF": "XGBoost",
    "TFT ACF": "TFT",
}
for tr in subfig.data:
    if tr.name in rename:
        tr.name = rename[tr.name]
subfig.update_xaxes(title_text="Lag (hours)", linecolor="lightgrey", mirror=True, dtick=12, showgrid=True)
subfig.update_yaxes(title_text="ACF", linecolor="lightgrey", mirror=True, showgrid=True)
pio.write_html(
    subfig, "subfig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
subfig

In [ ]:
XGB_corr = [
    corr_F1.loc['LSTM', 'XGBoost'], 
    corr_F2.loc['LSTM', 'XGBoost'],
    corr_F3.loc['LSTM', 'XGBoost'],
    corr_F4.loc['LSTM', 'XGBoost'],
    corr_F5.loc['LSTM', 'XGBoost'],
]
SARIMAX_corr = [
    corr_F1.loc['LSTM', 'SARIMAX'], 
    corr_F2.loc['LSTM', 'SARIMAX'],
    corr_F3.loc['LSTM', 'SARIMAX'],
    corr_F4.loc['LSTM', 'SARIMAX'],
    corr_F5.loc['LSTM', 'SARIMAX'],
]
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, 6)),
    y=XGB_corr,
    mode='lines+markers',
    name='LSTM vs XGBoost'
))
fig.add_trace(go.Scatter(
    x=list(range(1, 6)),
    y=SARIMAX_corr,
    mode='lines+markers',
    name='LSTM vs SARIMAX'
))
fig.update_layout(
    title=None,
    xaxis_title='Series ID',
    yaxis_title='Residual Correlation',
    template='plotly_white',
    height=400,
    width=750,
    legend=dict(
        x=0.0,
        y=1.0,
        bgcolor="rgba(255,255,255,0.7)",  
        title=""
    ),
    margin=dict(l=60, r=20, t=20, b=60),
)
fig.update_xaxes(linecolor="lightgrey", mirror=True)
fig.update_yaxes(linecolor="lightgrey", mirror=True)
fig.update_xaxes(
    tickvals=[1, 2, 3, 4, 5],
    ticktext=['F1', 'F2', 'F3', 'F4', 'F5']
)
pio.write_html(
    fig, "fig.html",
    include_plotlyjs="inline", full_html=True,
    config={"toImageButtonOptions": {"format": "svg"}}
)
fig.show()